In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting map library

print('Folium installed')
print('All Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
All Libraries imported.


In [3]:
CLIENT_ID='NSWC0SRL1NVQY551AP3I0RAQPIFKF3PXC3XBYGT1DEGC0CJM'
CLIENT_SECRET='1EDCF3ZSYUHAO2A03AOK0QEIIWM1UGSL3XBSYKTV4LYHAWV2'
latitude=43.6427
longitude=-79.3888
VERSION=20200401
search_query = 'hotels'
radius = 10000 # 10 KM
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
result_hotels = requests.get(url).json()
result_hotels

{'meta': {'code': 200, 'requestId': '5e970e5cb1cac0001bbdf5fe'},
 'response': {'venues': [{'id': '53357710498e20817350cfb4',
    'name': 'Delta Hotels by Marriott Toronto',
    'location': {'address': '75 Lower Simcoe Street',
     'lat': 43.6428819,
     'lng': -79.3839491,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.6428819,
       'lng': -79.3839491}],
     'distance': 391,
     'postalCode': 'M5J 3A6',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['75 Lower Simcoe Street',
      'Toronto ON M5J 3A6',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1fa931735',
      'name': 'Hotel',
      'pluralName': 'Hotels',
      'shortName': 'Hotel',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1586957999',
    'hasPerk': False},
   {'id': '4c798164bd346dcbb2baf5ef',
    'name': "Soho Metropo

In [4]:
# assign relevant part of JSON to hotel
hotels = result_hotels['response']['venues']

# tranform hotels into a dataframe
df_hotels = json_normalize(hotels)
df_hotels.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,53357710498e20817350cfb4,Delta Hotels by Marriott Toronto,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1586957999,False,75 Lower Simcoe Street,43.642882,-79.383949,"[{'label': 'display', 'lat': 43.6428819, 'lng'...",391,M5J 3A6,CA,Toronto,ON,Canada,"[75 Lower Simcoe Street, Toronto ON M5J 3A6, C...",NaN
1,4c798164bd346dcbb2baf5ef,Soho Metropolitan Hotel's Health Club,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1586957999,False,318 Wellington Street West,43.645129,-79.392565,"[{'label': 'display', 'lat': 43.64512881281113...",406,M5V 3T4,CA,Toronto,ON,Canada,"[318 Wellington Street West, Toronto ON M5V 3T...",NaN
2,541c283f498eb661ff11f1c0,FRHI Hotels & Resorts,"[{'id': '4bf58dd8d48988d174941735', 'name': 'C...",v-1586957999,False,155 Wellington Street West,43.645677,-79.385340,"[{'label': 'display', 'lat': 43.64567660611423...",432,M5V 0C3,CA,Toronto,ON,Canada,"[155 Wellington Street West, Toronto ON M5V 0C...",NaN
3,4dffb594aeb7fa1f7b053dd2,Fairmont Hotels & Resort Corporate Office,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1586957999,False,"155 Wellington St., W. Suite 3300",43.645550,-79.383256,"[{'label': 'display', 'lat': 43.64554991465454...",547,M5V 0C3,CA,Toronto,ON,Canada,"[155 Wellington St., W. Suite 3300, Toronto ON...",NaN
4,50d20e9745b05cd54c363631,Delta Hotels & Resorts Corporate Office,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",v-1586957999,False,"77 King Street W, Suite 2300",43.648063,-79.381993,"[{'label': 'display', 'lat': 43.64806290554908...",810,M5K 1G8,CA,Toronto,ON,Canada,"[77 King Street W, Suite 2300, Toronto ON M5K ...",NaN


In [5]:
# keep only columns that include hotel name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_hotels.columns if col.startswith('location.')] + ['id']
df_hotels_filtered = df_hotels.loc[:, filtered_columns]

# function that extracts the category of the hotels
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
df_hotels_filtered['categories'] = df_hotels_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_hotels_filtered.columns = [column.split('.')[-1] for column in df_hotels_filtered.columns]

pd.DataFrame(df_hotels_filtered)
df_hotels_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Delta Hotels by Marriott Toronto,Hotel,75 Lower Simcoe Street,43.642882,-79.383949,"[{'label': 'display', 'lat': 43.6428819, 'lng'...",391,M5J 3A6,CA,Toronto,ON,Canada,"[75 Lower Simcoe Street, Toronto ON M5J 3A6, C...",NaN,53357710498e20817350cfb4
1,Soho Metropolitan Hotel's Health Club,Gym / Fitness Center,318 Wellington Street West,43.645129,-79.392565,"[{'label': 'display', 'lat': 43.64512881281113...",406,M5V 3T4,CA,Toronto,ON,Canada,"[318 Wellington Street West, Toronto ON M5V 3T...",NaN,4c798164bd346dcbb2baf5ef
2,FRHI Hotels & Resorts,Coworking Space,155 Wellington Street West,43.645677,-79.385340,"[{'label': 'display', 'lat': 43.64567660611423...",432,M5V 0C3,CA,Toronto,ON,Canada,"[155 Wellington Street West, Toronto ON M5V 0C...",NaN,541c283f498eb661ff11f1c0
3,Fairmont Hotels & Resort Corporate Office,Office,"155 Wellington St., W. Suite 3300",43.645550,-79.383256,"[{'label': 'display', 'lat': 43.64554991465454...",547,M5V 0C3,CA,Toronto,ON,Canada,"[155 Wellington St., W. Suite 3300, Toronto ON...",NaN,4dffb594aeb7fa1f7b053dd2
4,Delta Hotels & Resorts Corporate Office,Office,"77 King Street W, Suite 2300",43.648063,-79.381993,"[{'label': 'display', 'lat': 43.64806290554908...",810,M5K 1G8,CA,Toronto,ON,Canada,"[77 King Street W, Suite 2300, Toronto ON M5K ...",NaN,50d20e9745b05cd54c363631
5,OMNI🌺 Hotels & Resorts,Building,NaN,43.649510,-79.376242,"[{'label': 'display', 'lat': 43.64950973623707...",1264,NaN,CA,NaN,NaN,Canada,[Canada],NaN,561b9bfa498e44b8783f2065
6,Soho Hotel Salon Daniel,Salon / Barbershop,86 Blue Jays Way,43.645279,-79.391894,"[{'label': 'display', 'lat': 43.64527893815516...",380,NaN,CA,Toronto,ON,Canada,"[86 Blue Jays Way, Toronto ON, Canada]",NaN,4eee35b446907303bfe3fd40
7,Hollywood Ball at The King Edward Hotel Sponso...,Nightlife Spot,NaN,43.649358,-79.376507,"[{'label': 'display', 'lat': 43.649358, 'lng':...",1236,NaN,CA,Toronto,ON,Canada,"[King & Victoria, Toronto ON, Canada]",King & Victoria,4dc4a1f2c65b89d3ca5a79e0
8,Skyline Hotels & Resorts,Office,90 Eglinton Ave E,43.707578,-79.395640,"[{'label': 'display', 'lat': 43.70757799282385...",7243,M4P 2Y5,CA,Toronto,ON,Canada,[90 Eglinton Ave E (Yonge St. & Eglinton Ave.)...,Yonge St. & Eglinton Ave.,4e67b8e7d16433b90078385a
9,Hotel Shames,Hotel Bar,NaN,43.654970,-79.421245,"[{'label': 'display', 'lat': 43.65497, 'lng': ...",2948,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4fdcf92ee4b070b1a56e8716


In [6]:
df_hotels_filtered = df_hotels_filtered[df_hotels_filtered.categories.isin(['Hotel', 'Office'])]
df_hotels_filtered.categories = 'Hotel'
df_hotels_filtered.reset_index(drop=True, inplace=True)
df_hotels_filtered

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Delta Hotels by Marriott Toronto,Hotel,75 Lower Simcoe Street,43.642882,-79.383949,"[{'label': 'display', 'lat': 43.6428819, 'lng'...",391,M5J 3A6,CA,Toronto,ON,Canada,"[75 Lower Simcoe Street, Toronto ON M5J 3A6, C...",NaN,53357710498e20817350cfb4
1,Fairmont Hotels & Resort Corporate Office,Hotel,"155 Wellington St., W. Suite 3300",43.645550,-79.383256,"[{'label': 'display', 'lat': 43.64554991465454...",547,M5V 0C3,CA,Toronto,ON,Canada,"[155 Wellington St., W. Suite 3300, Toronto ON...",NaN,4dffb594aeb7fa1f7b053dd2
2,Delta Hotels & Resorts Corporate Office,Hotel,"77 King Street W, Suite 2300",43.648063,-79.381993,"[{'label': 'display', 'lat': 43.64806290554908...",810,M5K 1G8,CA,Toronto,ON,Canada,"[77 King Street W, Suite 2300, Toronto ON M5K ...",NaN,50d20e9745b05cd54c363631
3,Skyline Hotels & Resorts,Hotel,90 Eglinton Ave E,43.707578,-79.395640,"[{'label': 'display', 'lat': 43.70757799282385...",7243,M4P 2Y5,CA,Toronto,ON,Canada,[90 Eglinton Ave E (Yonge St. & Eglinton Ave.)...,Yonge St. & Eglinton Ave.,4e67b8e7d16433b90078385a
4,Four Seasons Hotels and Resorts,Hotel,NaN,43.728090,-79.350137,"[{'label': 'display', 'lat': 43.72808982700887...",10002,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,4cdc2ad5d4ecb1f781d97d48
5,Four Seasons Hotel Toronto,Hotel,60 Yorkville Avenue,43.671796,-79.389457,"[{'label': 'display', 'lat': 43.67179643581682...",3239,M4W 0A4,CA,Toronto,ON,Canada,"[60 Yorkville Avenue (btwn Bay & Yonge), Toron...",btwn Bay & Yonge,4ad4c05bf964a520b0f520e3


In [7]:
df_hotels_filtered.drop(['crossStreet', 'labeledLatLngs', 'postalCode', 'state', 'country', 'city'], axis=1, inplace=True)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
df_hotels_filtered

,name,categories,address,lat,lng,distance,cc,formattedAddress,id
0,Delta Hotels by Marriott Toronto,Hotel,75 Lower Simcoe Street,43.642882,-79.383949,391,CA,"[75 Lower Simcoe Street, Toronto ON M5J 3A6, C...",53357710498e20817350cfb4
1,Fairmont Hotels & Resort Corporate Office,Hotel,"155 Wellington St., W. Suite 3300",43.645550,-79.383256,547,CA,"[155 Wellington St., W. Suite 3300, Toronto ON...",4dffb594aeb7fa1f7b053dd2
2,Delta Hotels & Resorts Corporate Office,Hotel,"77 King Street W, Suite 2300",43.648063,-79.381993,810,CA,"[77 King Street W, Suite 2300, Toronto ON M5K ...",50d20e9745b05cd54c363631
3,Skyline Hotels & Resorts,Hotel,90 Eglinton Ave E,43.707578,-79.395640,7243,CA,[90 Eglinton Ave E (Yonge St. & Eglinton Ave.)...,4e67b8e7d16433b90078385a
4,Four Seasons Hotels and Resorts,Hotel,NaN,43.728090,-79.350137,10002,CA,"[Toronto ON, Canada]",4cdc2ad5d4ecb1f781d97d48
5,Four Seasons Hotel Toronto,Hotel,60 Yorkville Avenue,43.671796,-79.389457,3239,CA,"[60 Yorkville Avenue (btwn Bay & Yonge), Toron...",4ad4c05bf964a520b0f520e3


In [11]:
allhotels_map = folium.Map(location=[latitude, longitude], zoom_start=10) # generate map centred around the New York City


coordinates = [
    [43.772692,-79.504171],
    [43.740158,-79.216718],
    [43.638427,-79.382013],
    [43.648696,-79.562161],
    [43.772692,-79.504171]
]

my_PolyLine=folium.PolyLine(locations=coordinates,weight=5)
allhotels_map.add_child(my_PolyLine)

# add the Hotels as blue circle markers
for lat, lng, label in zip(df_hotels_filtered.lat, df_hotels_filtered.lng, df_hotels_filtered.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(allhotels_map)

# display map
allhotels_map